#### Notes:

##### Reference Scripts:
- https://github.com/ranayalcink/selenium_web_scraper/blob/main/glassdoor_scraper.py
- https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb
- https://github.com/Khushi130404/Job-Scraper/blob/main/Multi_Glassdoor/Scrape_Glassdoor2.py
  
##### This script uses ChromeDriver to scrape. You can read how to download and get it set up below:

- https://developer.chrome.com/docs/chromedriver/get-started
- https://developer.chrome.com/docs/chromedriver/downloads 

In [ ]:
#I copy and pasted this from the website above
#You can run this to ensure you've properly set up Chrome Driver

import time
from selenium import webdriver

driver = webdriver.Chrome('/path/to/chromedriver')  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/')
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()

#### Glassdoor Jobs Scraper:

In [41]:
#Load necessary packages:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time, random, csv
import pandas as pd

In [49]:
# NEEDED AMENDS:

# 1. CHANGE FROM num_pages to NUMBER OF JOB CARDS
# 2. FIX csv EXPORTER
# 3. CHANGE EXPORTED FILE NAME TO INCLUDE LOCATION AND JOB NUMBERS AS WELL
# 4. EITHER REMOVE COMPANY DETAILS OR NEED TO CLICK INTO COMPANY PAGE AND EXTRACT DETAILS THERE


def fetch_jobs(keyword, num_pages, location):
    options = Options()
    options.add_argument("window-size=1920,1080")
    # Initialize the driver (ChromeDriverManager installs/upgrades the driver automatically)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Open Glassdoor's main job search page
    driver.get("https://www.glassdoor.co.uk/Job/index.htm")
    time.sleep(2)
    #close_popups() # Check for pop-ups after page loads
    
    # Enter search criteria (make sure these IDs are up-to-date)
    search_input = driver.find_element(By.ID, "searchBar-jobTitle")
    location_input = driver.find_element(By.ID, "searchBar-location")
    location_input.clear()
    search_input.send_keys(keyword)
    location_input.send_keys(location)
    location_input.send_keys(Keys.RETURN)
    time.sleep(3)
    
    all_data = []  # List to store job data

    def random_sleep():
        time.sleep(random.uniform(10, 25))
        
    def click_load_more():
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-test='load-more']"))
            )
            load_more_button.click()
            time.sleep(5)  # wait for new jobs to load
            print("Clicked 'Load More' button")
            return True
        except Exception:
            print("No 'Load More' button found or error occurred")
            return False
###################################################################################################################################################
    ######### CLOSE POP UPS FUNCTIONS: ########
    #Close Create Job Alert Button: class="icon-button_IconButton__nMTOc" ; type="button"
    #Close log-in pop up button:   class="CloseButton" ; type="button"

    def close_popups():
        try:
            # Close login pop-up if it appears
            login_popup = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.CloseButton[type='button']"))
            )
            login_popup.click()
            print("Closed login pop-up")
            time.sleep(2)
        except Exception:
            print("No login pop-up detected")

###################################################################################################################################################
    def scrape_page():
        nonlocal all_data
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Select all job cards – update the selector if necessary
        job_cards = soup.select("a[data-test='job-link']")
        for job_card in job_cards:
            try:
                job_link = job_card.get('href')
                if job_link and job_link.startswith("/"):
                    job_link = "https://www.glassdoor.co.uk" + job_link
                print(f"Visiting job link: {job_link}")
                driver.get(job_link)
                close_popups() # Check for and close pop-ups
                random_sleep()
                
                # Attempt to click the "Show more" button to reveal full job description
                try:
                    show_more_button = driver.find_element(By.CSS_SELECTOR, '[data-test="show-more-cta"]')
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", show_more_button)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", show_more_button)
                    print("Clicked 'Show more' button successfully.")
                except Exception:
                    print("No 'Show more' button found.")
                    
                #Get the updated page source after clicking "Show more"
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                #Job Title
                title = soup.select_one("h1.heading_Heading__BqX5J")
                title = title.get_text(strip=True) if title else "No Title Available"

                #Company Name
                company = soup.select_one("h4.heading_Heading__BqX5J")
                company = company.get_text(strip=True) if company else "No Company Name Available"

                #Job Location
                location_elem = soup.select_one("[data-test='location']")
                location_text = location_elem.get_text(strip=True) if location_elem else "No Location Available"

                #Salary Estimate
                salary_elem = soup.select_one("[data-test='detailSalary']")
                salary_text = salary_elem.get_text(strip=True) if salary_elem else "Not Disclosed"
             
                # Looks for bullet point lists from within the job description. Will extract the list as well as the title for the list
                # Will extract up to three lists
                job_desc_div = soup.find("div", class_="JobDetails_jobDescription__uW_fK JobDetails_showHidden__C_FOA")
                descriptions = [
                    f"{ul.find_previous(['h3', 'h4', 'strong', 'b', 'p']).get_text(strip=True) if ul.find_previous(['h3', 'h4', 'strong', 'b', 'p']) else f'Description {idx+1}'}:\n{' | '.join([li.get_text(strip=True) for li in ul.find_all('li')])}"
                    for idx, ul in enumerate(job_desc_div.find_all("ul")[:3])
                ] if job_desc_div else ["No job description available"]

###################################################################################################################################################

                #NOT LOGGED IN SO WONT BE ABLE TO ACCESS THIS DATA
                
                ######### SCRAPE COMPANY DETAILS: ########
                #Extract company overview details into own dictionary
                company_info = {}
                
                for item in soup.select(".JobDetails_overviewItem__cAsry"):
                    label_elem = item.select_one(".JobDetails_overviewItemLabel__KjFln")
                    value_elem = item.select_one(".JobDetails_overviewItemValue__xn8EF")
                
                    label_text = label_elem.get_text(strip=True) if label_elem else None
                    value_text = value_elem.get_text(strip=True) if value_elem else "No Data Available"
                
                    if label_text:
                        company_info[label_text] = value_text  # Stores in dictionary
                
                #Company Size: (eg. 1001 to 5000 Employees)
                #company_size = soup.select_one("[class='JobDetails_overviewItemValue__xn8EF']")
                #company_size = company_size.get_text(strip=True) if company_size else "No Company Size Available"

                #Labels & Example Data:
                #Founded (eg. 2014)
                #Type (eg. Company - Public)
                #Industry (eg. Internet & Web Services)
                #Sector (eg. Information Technology)
                #Revenue (eg. Unknown / Non-Applicable)

###################################################################################################################################################
                job_data = {
                    "Job Title": title,
                    "Company Name": company,
                    #"Company Size": company_size
                    #"Company Founded": company_founded
                    #"Company Type": company_type
                    #"Industry": industry
                    #"Sector": sector
                    #"Company Revenue": revenue
                    "Location": location_text,
                    "Salary": salary_text,
                    "Job Description 1": descriptions[0] if len(descriptions) > 0 else "No Description 1 Available",
                    "Job Description 2": descriptions[1] if len(descriptions) > 1 else "No Description 2 Available",
                    "Job Description 3": descriptions[2] if len(descriptions) > 2 else "No Description 3 Available",
                }
                
                # Merge company info dictionary into job_data
                job_data.update(company_info)

                # Append to final dataset
                all_data.append(job_data)
                
                driver.back()  # Return to the results list
                time.sleep(5)
            except Exception as e:
                print(f"Error processing job card: {e}")
                driver.back()
                time.sleep(5)

    # Iterate over the number of pages specified
    current_page = 1
    while current_page <= num_pages:
        print(f"Scraping page {current_page}")
        scrape_page()
        
        # If there's a "Load More" button, try clicking it until it's no longer available
        while click_load_more():
            scrape_page()
        
        #Try navigating to the next page if it exists.
        try:
            next_page_button = driver.find_element(By.CSS_SELECTOR, ".button_Button__MlD2g.button-base_Button__knLaX")
            next_page_button.click()
            time.sleep(5)
        except Exception as e:
            print(f"Could not navigate to next page: {e}")
            break
        
        current_page += 1

    driver.quit()
    return all_data
    
    # Save data to CSV. Should save to same directory as script (hopefully)
    
    if jobs:
        csv_file = f"{keyword}.csv"  # f-string method
        keys = jobs[0].keys()
        with open(csv_file, "w", newline="", encoding="utf-8") as output_file:
            dict_writer = csv.DictWriter(output_file, fieldnames=keys)
            dict_writer.writeheader()
            dict_writer.writerows(jobs)
        print(f"Scraping complete. Data saved to {csv_file}.")
    else:
        print("No job data was scraped.")


In [51]:
test = fetch_jobs("data scientist", 1, "London")

Scraping page 1
Visiting job link: https://www.glassdoor.co.uk/partner/jobListing.htm?pos=101&ao=1110586&s=58&guid=00000195bd9c024a80ac9c8c14edf20d&src=GD_JOB_AD&t=SR&vt=w&cs=1_2de686c9&cb=1742642873255&jobListingId=1009682111352&cpc=2CAED5C921A5F994&jrtk=5-yul1-0-1imupo0k72dih000-ef18d3efa1b03cd6---6NYlbfkN0AcrKzCeIcmmFjCJ4pdhmBCCVpanUW4iHjUPRj5vqJuLyLx3BNbgVIjbuyFlzVTOtkr6_NB9cTUtvB3S8eRNDgK-jP1xCCr87TZPT_2qXzcNRcJktC0MyvQfbqRn-6pj37Cd9rG0eTkM--3GaozFUzorxdonmr1abgftYoRVWb2zzMN6-kFHtLnePSuny6CncqgERtdHIbjgMh52HTKWTJvSYcPd5TQai_nTwFv3ASwPb8SgAEddjSXhzZ0bQzCkf0bHCjmAzA2lRyBEfvPwryqxLjv6PLymx0DITAIK8jpimrnrxQX9--I6BqBh91hnSMKZSjW7AssIBsbhW49ZNYe9Jzhd2Q7xphcEY_uF-l4A_p_WQBSNESZdt1gOHfK8mvR1eY8smymDznxuJLqKtaWJ1uXd5l4Bomm3R7ECxFKUsfG-4u4Stlh4Q4mVXbHINcoAQ5VGZGWfcTjWtLHNTAGpNEmSGpkTRq3r0aDHlSZjvKK9rAirM6ydLbygQe9dCxaxM9Fi1xzO_vo0007mOs1SddAgJbDlUc-6VSDKkuPLeXgWzNUvMIYNCmXFBLahLiXTRqj4bwTbB0XsBrblAhTXVzAsePc-J11EpZSZc5LZgSYdR9cS60-0PcQUaLJGP8oC6b36ZqVTWaOxj1jYsXb9jpBaQPwezNQjllFVLWFX6AcaOTB

In [67]:
test

[{'Job Title': 'Snr Data Scientist - GenAI',
  'Company Name': 'Coca-Cola Europacific Partners',
  'Location': 'Uxbridge, England',
  'Salary': '£54K - £79K(Glassdoor Est.)',
  'Job Description 1': 'Key Responsibilities::\nDevelop and deploy GenAI models tailored for commercial applications. | Implement RAG techniques and other generative AI strategies to enhance business processes. | Ensure production-grade deployment using MLOps and LLMOps best practices. | Collaborate with cross-functional teams to align AI initiatives with business goals. | Stay updated on advancements in Generative AI and large language model frameworks.',
  'Job Description 2': 'Qualifications::\nMaster’s degree in Artificial Intelligence, Computer Science, or a related field; a PhD is preferred. | Expertise in Generative AI and large language models (e.g., Hugging Face, OpenAI APIs). | Experience with MLOps and LLMOps for scalable model deployment. | Proficiency in Python and cloud-based data platforms (e.g., Da

In [69]:
# Convert the dictionary to a DataFrame
df = pd.DataFrame(test)

# Export the DataFrame to a CSV file
df.to_csv('job_descriptions_test1.csv', index=False)

In [71]:
print("Job Data: ", job_data)

NameError: name 'job_data' is not defined